# 検定のためのサンプルサイズの決め方

母比率$p$の95%信頼区間は、標本比率$\hat{p}$を用いて

$$
\widehat{p}-1.96 \times \sqrt{\frac{\widehat{p}(1-\widehat{p})}{n}} \leq p \leq \widehat{p}+1.96 \times \sqrt{\frac{\widehat{p}(1-\widehat{p})}{n}}
$$

と求めることができる

区間の幅部分が両側合計で$w$以下にしたい場合は

$$
2 \times 1.96 \times \sqrt{\frac{\widehat{p}(1-\widehat{p})}{n}} \leq w
$$

を解けばよいので


$$
\sqrt{n} \geq 2 \times 1.96 \times \sqrt{\widehat{p}(1-\widehat{p})} \times \frac{1}{w}
$$



$$
n \geq (2 \times 1.96 \times \sqrt{\widehat{p}(1-\widehat{p})} \times \frac{1}{w})^2
$$


となる

例えば、事前調査により$\hat{p} = 0.1$であることがわかっていて、幅$w=0.01$で95%信頼区間を求めたい場合は

In [38]:
from math import sqrt

w = 0.01
p_hat = 0.1

n = int( (2 * 1.96 * sqrt(p_hat * (1 - p_hat)) / w) ** 2 )
print(f"{n=}")

n=13829


In [43]:
# 信頼区間の計算
from scipy.stats import norm, sem

mu = p_hat
lower, upper = norm.interval(confidence=0.95, loc=mu, scale=sqrt((p_hat * (1 - p_hat)) / n))
width = (mu - lower) + (upper - mu)
print(f"[{lower:.3g}, {upper:.3g}], width={width:.1g}")

[0.095, 0.105], width=0.01


In [ ]:
# 乱数生成してシミュレーション
import numpy as np
np.random.seed(0)
x = np.random.binomial(n=1, p=p_hat, size=int(n))

from scipy.stats import norm, sem
lower, upper = norm.interval(confidence=0.95, loc=x.mean(), scale=sem(x))
width = (x.mean() - lower) + (upper - x.mean())
print(f"[{lower:.3g}, {upper:.3g}], width={width:.1g}")

## 参考

- [21-4. 必要なサンプルサイズ1 | 統計学の時間 | 統計WEB](https://bellcurve.jp/statistics/course/9129.html?srsltid=AfmBOor_VrX0VRNoaDn20rzg-4p3TdUMNflYCO7EGUwvrOtqo6LQ94bw)
- [Wang, H., & Chow, S. C. (2014). Sample Size Calculation for Comparing Variabilities. Wiley StatsRef: Statistics Reference Online. John Wiley & Sons, Ltd.](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=77a4e5a32157ddd2c8ccab43093e7a7602e6938f)
    - ちゃんとやる場合にはこれ参考になりそう
- [サンプルサイズの決め方 (統計ライブラリー) | 永田 靖 |本 | 通販 | Amazon](https://www.amazon.co.jp/dp/4254126654)
